# Visualising Single Stage Performance

If we get the scores data for a single stage, we can visualise the performances within the stage at each waypoint.

In [1]:
from dakar_rallydj.getter import DakarAPIClient

def custom_key_fn(request, ignored_parameters=None, match_headers=False, serializer=None, **request_kwargs):
    return request.url.split("api/")[-1]  # Store the plain URL as the key

dakar = DakarAPIClient(
    use_cache=True,
    backend='sqlite',
    cache_name='dakar_cache_2025',
    expire_after=-1,  # Never expire
    key_fn=custom_key_fn
)

In [2]:
# Load in the data from the Dakar cached live data site

long_results_df, long_results2_df, teams_df, competitors_df = dakar.get_scores()

In [ ]:
long_results_df

,_id,team.bib,type,waypoint,metric,value_0,value_1,year,category,stage
0,lastScore-2025-A-1-427,427,cg,01216,position,128,128,2025,A,1
1,lastScore-2025-A-1-634,634,cg,01216,position,194,194,2025,A,1
2,lastScore-2025-A-1-330,330,cg,01216,position,53,53,2025,A,1
3,lastScore-2025-A-1-243,243,cg,01216,position,60,60,2025,A,1
4,lastScore-2025-A-1-404,404,cg,01216,position,65,65,2025,A,1
...,...,...,...,...,...,...,...,...,...,...
12569,lastScore-2025-A-1-501,501,cg,01227,position,1,1,2025,A,1
12680,lastScore-2025-A-1-500,500,cg,01227,absolute,13622,13622,2025,A,1
12770,lastScore-2025-A-1-501,501,cg,01227,absolute,13510,13510,2025,A,1
12881,lastScore-2025-A-1-500,500,cg,01227,relative,112,112,2025,A,1


In [4]:
from dakar_rallydj.enrichers import derive_clazz_metadata

clazz_df = dakar.get_clazz()
groups_df = dakar.get_groups()

teams_df = derive_clazz_metadata(teams_df, clazz_df, groups_df)
teams_df

,team.bib,reference,categoryClazz,clazz_label,tinyLabel,label,color,group_label
0,200,2025-A-T1-+,2025-A-T1,T1+: Prototype Cross-Country Cars 4x4,ULT,T1,#EBBC4E,Ultimate
1,201,2025-A-T1-+,2025-A-T1,T1+: Prototype Cross-Country Cars 4x4,ULT,T1,#EBBC4E,Ultimate
2,202,2025-A-T1-+,2025-A-T1,T1+: Prototype Cross-Country Cars 4x4,ULT,T1,#EBBC4E,Ultimate
3,203,2025-A-T1-+,2025-A-T1,T1+: Prototype Cross-Country Cars 4x4,ULT,T1,#EBBC4E,Ultimate
4,204,2025-A-T1-+,2025-A-T1,T1+: Prototype Cross-Country Cars 4x4,ULT,T1,#EBBC4E,Ultimate
...,...,...,...,...,...,...,...,...
196,646,2025-A-T5-2,2025-A-T5,T5.2: Race Service Assistance Trucks,TRK,T5,#2D2926,Truck
197,647,2025-A-T5-2,2025-A-T5,T5.2: Race Service Assistance Trucks,TRK,T5,#2D2926,Truck
198,648,2025-A-T5-2,2025-A-T5,T5.2: Race Service Assistance Trucks,TRK,T5,#2D2926,Truck
199,649,2025-A-T5-2,2025-A-T5,T5.2: Race Service Assistance Trucks,TRK,T5,#2D2926,Truck


We probably just want to grab vehicles by class. So let's grab `team.bib` values for vehicles in a particul;ar class, and then use those values to filter the results.

In [6]:
CLASS="ULT"

teams = teams_df[teams_df["tinyLabel"]==CLASS]["team.bib"]

long_results_df[long_results_df["team.bib"].isin(teams)]

,_id,team.bib,type,waypoint,metric,value_0,value_1,year,category,stage
3,lastScore-2025-A-1-243,243,cg,01216,position,60,60,2025,A,1
6,lastScore-2025-A-1-213,213,cg,01216,position,36,36,2025,A,1
7,lastScore-2025-A-1-261,261,cg,01216,position,189,189,2025,A,1
13,lastScore-2025-A-1-204,204,cg,01216,position,3,3,2025,A,1
22,lastScore-2025-A-1-230,230,cg,01216,position,8,8,2025,A,1
...,...,...,...,...,...,...,...,...,...,...
11846,lastScore-2025-A-1-216,216,cs,01ASS,relative,1540,1540,2025,A,1
11847,lastScore-2025-A-1-215,215,cs,01ASS,relative,1508,1508,2025,A,1
11848,lastScore-2025-A-1-223,223,cs,01ASS,relative,9106,9106,2025,A,1
11854,lastScore-2025-A-1-212,212,cs,01ASS,relative,133,133,2025,A,1


Creating arbitrarily complex bindings across multiple dataframes, as well as filtering rows within a particular dataframe, is possible using *pandas* constructions, but it is often more convenient to use the SQL database query language over records that are contained in a relational database table.

In [15]:
from sqlite_utils import Database
import pandas as pd

db = Database("dakar_results_2025.sqlite")

In [16]:
pd.DataFrame(db.query("select * from long_results2  limit 2"))

,_id,team_bib,metric,value_0,value_1,year,category,stage
0,lastScore-2025-M-12-19,19,position,9,9,2025,M,12
1,lastScore-2025-M-12-82,82,position,79,79,2025,M,12


In [13]:
import pandas as pd

q = """
SELECT  DISTINCT(type) FROM long_results AS l JOIN results_teams AS t ON l.team_bib=t.team_bib WHERE t.tinylabel="ULT" ;
"""
pd.DataFrame(db.query(q))

,type
0,cg
1,cs
